In [7]:
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

class BearingDataset(Dataset):
    def __init__(self, vibration, rul, seq_length, device):
        self.vibration = vibration
        self.rul = rul
        self.seq_length = seq_length
        self.device = device

    def __len__(self):
        return len(self.rul) - self.seq_length + 1

    def __getitem__(self, idx):
        x = self.vibration[idx:idx + self.seq_length]
        y = self.rul[idx + self.seq_length - 1]

        x = torch.tensor(x, dtype=torch.float32).to(self.device)
        y = torch.tensor(y, dtype=torch.float32).to(self.device)

        return x, y

In [25]:
from lib.dataset import *

criterion = nn.BCEWithLogitsLoss()
data_paths = get_bearing_paths("./datasets")
device = torch.device("cuda")
batch_size = 256
seq_length = 128

In [26]:
train_tasks = []
val_tasks = []
for i in range(len(data_paths)-12):
    df = load_data(data_paths[i] + "/")
    df = preprocess_data(df)

    vibration = df[['Horizontal_vibration_signals', 'Vertical_vibration_signals']].values
    rul = df['Normalized_RUL'].values

    if i in [0,5,10]:
        val_tasks.append(dataloader)
        dataset = BearingDataset(vibration, rul, seq_length, device)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    else:
        train_tasks.append(dataloader)
        dataset = BearingDataset(vibration, rul, seq_length, device)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Load Dataset: ./datasets\35Hz12kN\Bearing1_3/: 100%|██████████| 158/158 [00:03<00:00, 47.34it/s]


In [ ]:
import torch.optim as optim
from lib.model import Generator, Discriminator
from lib.engine import train_gan_model

generator = Generator(2, 256)
discriminator = Discriminator(2, 256)

lr = 0.01
g_optimizer = optim.AdamW(generator.parameters(), lr=lr)
d_optimizer = optim.AdamW(discriminator.parameters(), lr=lr)

train_gan_model(
    train_tasks=train_tasks,
    generator=generator,
    discriminator=discriminator,
    criterion=criterion,
    g_optimizer = g_optimizer,
    d_optimizer = d_optimizer,
    num_epochs=3,
    device = device,
    log_file="sample_gan.log",
    gan_model_path="./models/sample_gan_model.pth",
    dis_model_path="./models/sample_dis_model.pth"
)


In [ ]:
from lib.engine import generate_augmented_data

num_augmented_samples = batch_size

for batch in val_tasks[0]:
    x, y = batch
    x = generate_augmented_data(generator, x, num_augmented_samples, device)
    y = y.repeat(1+num_augmented_samples // len(y))

    break